# NAMs

In [1]:
import pandas as pd
import numpy as np

file_path = r"A:\Seismic_data_analysis\Seismic-Data-analysis\Data\NGAsub_MegaFlatfile_RotD50_050_R211022_public.xlsx"
df = pd.read_excel(file_path)

df.head()

,NGAsubRSN,DatabaseRegion,NGAsubEQID,NGAsubSSN,Earthquake_Name,YEAR,MODY,HRMN,Earthquake_Magnitude,Hypocenter_Latitude_deg,...,T8pt500S,T9pt000S,T9pt500S,T10pt000S,T11pt000S,T12pt000S,T13pt000S,T14pt000S,T15pt000S,T20pt000S
0,1000001,Alaska,1000001,1000001,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000082,0.000095,0.000103,0.000125,0.000138,0.000109,0.000089,0.000064,0.000053,0.000061
1,1000002,Alaska,1000001,1000002,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000415,0.000395,0.000405,0.000390,0.000336,0.000332,0.000263,0.000215,0.000187,0.000119
2,1000003,Alaska,1000001,1000003,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000092,0.000083,0.000086,0.000082,0.000091,0.000078,0.000071,0.000066,0.000060,0.000055
3,1000004,Alaska,1000001,1000004,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000055,0.000053,0.000051,0.000055,0.000059,0.000058,0.000042,0.000035,0.000038,0.000028
4,1000005,Alaska,1000001,1000005,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000129,0.000116,0.000128,0.000122,0.000130,0.000119,0.000096,0.000135,0.000162,0.000060


In [37]:
df_fil = df[
    (df['Earthquake_Magnitude'] >= 4) & 
    (df['Rjb_km'] > 0) & 
    (df['Rjb_km'] <= 500) & 
    (df['Vs30_Selected_for_Analysis_m_s'] > 0) & 
    (df['Hypocenter_Depth_km'] > 0)
].copy()

# Inputs
X = pd.DataFrame()
X['M']         = df_fil['Earthquake_Magnitude']
X['logVs']     = np.log10(df_fil['Vs30_Selected_for_Analysis_m_s'])
X['logRrup']   = np.log10(df_fil['Rjb_km'])
X['Hyp_depth'] = df_fil['Hypocenter_Depth_km']
X["Rup"] = df_fil["Rjb_km"]

# Outputs
time_cols = [col for col in df_fil.columns if col.startswith('T') and col.endswith('S')]
output_cols = time_cols + ['PGA_g', 'PGV_cm_sec']

epsilon = 1e-8
y = np.log10(df_fil[output_cols].clip(lower=epsilon))

# Convert
X = X.to_numpy()
y = y.to_numpy()

print("Input shape:", X.shape)
print("Output shape:", y.shape)



Input shape: (55049, 5)
Output shape: (55049, 113)


In [38]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X = torch.tensor(X , dtype=torch.float32).to(device)
y = torch.from_numpy(y).float().to(device)

# If y is already tensor


perm = torch.randperm(X.shape[0] , device=device)

X = X[perm]
y = y[perm]

train_ratio = 0.8

n_train = int(train_ratio * X.shape[0])

X_train = X[:n_train]
y_train = y[:n_train]

X_test = X[n_train : ]
y_test = y[n_train : ]

from torch.utils.data import TensorDataset, DataLoader

batch_size = 256

train_dataset = TensorDataset(X_train, y_train)
test_dataset  = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [39]:
import torch.nn as nn

class model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            nn.Linear(5, 128),
            nn.ReLU(),
            
            nn.Linear(128, 256),
            nn.ReLU(),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            
            nn.Linear(128, 113)
            
            )
        
    def forward(self , X):
        return self.net(X)

model = model0().to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters() , lr = 1e-3)

In [42]:
epochs = 100

for epoch in range(epochs):
    
    model.train()
    train_loss = 0.0
    
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(xb)
        loss = criterion(outputs, yb)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    # Validation
    model.eval()
    test_loss = 0.0
    
    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            
            outputs = model(xb)
            loss = criterion(outputs, yb)
            test_loss += loss.item()
    
    test_loss /= len(test_loader)
    
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.6f} | Test Loss: {test_loss:.6f}")


Epoch 10/100 | Train Loss: 0.261561 | Test Loss: 0.240173
Epoch 20/100 | Train Loss: 0.246969 | Test Loss: 0.239373
Epoch 30/100 | Train Loss: 0.238205 | Test Loss: 0.257140
Epoch 40/100 | Train Loss: 0.225448 | Test Loss: 0.451559
Epoch 50/100 | Train Loss: 0.225316 | Test Loss: 0.263754
Epoch 60/100 | Train Loss: 0.219231 | Test Loss: 0.215414
Epoch 70/100 | Train Loss: 0.217768 | Test Loss: 0.237124
Epoch 80/100 | Train Loss: 0.219988 | Test Loss: 0.283157
Epoch 90/100 | Train Loss: 0.210384 | Test Loss: 0.211379
Epoch 100/100 | Train Loss: 0.212826 | Test Loss: 0.215337


In [43]:
from sklearn.metrics import r2_score

model.eval()
with torch.no_grad():
    preds = model(X_test).cpu().numpy()
    true  = y_test.cpu().numpy()

r2 = r2_score(true, preds)
print("R²:", r2)


R²: 0.6358232498168945
